In [ ]:
import os
import json
import csv
from collections import defaultdict

# Function to calculate IoU (Intersection over Union)
def calculate_iou(box1, box2):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2
    x_left = max(x1, x2)
    y_top = max(y1, y2)
    x_right = min(x1 + w1, x2 + w2)
    y_bottom = min(y1 + h1, y2 + h2)
    if x_right < x_left or y_bottom < y_top:
        return 0.0
    intersection_area = (x_right - x_left) * (y_bottom - y_top)
    box1_area = w1 * h1
    box2_area = w2 * h2
    iou = intersection_area / (box1_area + box2_area - intersection_area)
    return iou

# Function to process files based on ground truth and prediction JSONs
def process_files(gt_file, prediction_file):
    # Load ground truth annotations
    with open(gt_file, "r") as f:
        gt_data = json.load(f)

    # Load prediction annotations
    with open(prediction_file, "r") as f:
        pred_data = json.load(f)

    # Create defaultdict to store annotations by filename
    annotations = defaultdict(list)
    matched_predictions = set()

    # Index annotations by image_id
    gt_annotations_by_image = defaultdict(list)
    for ann in gt_data['annotations']:
        gt_annotations_by_image[ann['image_id']].append(ann)
    
    pred_annotations_by_image = defaultdict(list)
    for ann in pred_data:
        pred_annotations_by_image[ann['image_id']].append(ann)

    # Process each image in ground truth data
    for img in gt_data['images']:
        image_id = img['id']
        filename = img['file_name']
        img_width = img['width']
        img_height = img['height']

        # Fetch annotations for current image from both ground truth and predictions
        gt_annotations = gt_annotations_by_image[image_id]
        pred_annotations = pred_annotations_by_image[image_id]

        # Process each ground truth annotation
        for gt_obj in gt_annotations:
            gt_box = gt_obj['bbox']
            gt_obj_id = gt_obj['category_id']
            matched_pred = None
            max_iou = 0
            
            # Find the best matching prediction based on IoU
            for pred_obj in pred_annotations:
                iou = calculate_iou(gt_box, pred_obj['bbox'])
                if iou > max_iou:
                    max_iou = iou
                    matched_pred = pred_obj
            
            # If a matching prediction is found and IoU is sufficient
            if matched_pred and max_iou >= 0.5:
                # Use a tuple of relevant values as a unique identifier in matched_predictions
                pred_tuple = (
                    matched_pred['category_id'], 
                    tuple(matched_pred['bbox']),
                    matched_pred.get('score', None),
                    tuple(matched_pred['score_all'])
                )
                matched_predictions.add(pred_tuple)
                
                annotations[filename].append(
                    [filename, img_width, img_height, gt_obj_id] + gt_box + 
                    [matched_pred['category_id']] + matched_pred['bbox'] + 
                    [matched_pred.get('score', None)] + list(matched_pred['score_all'])  # Add score_all values
                )
            else:
                # Handle unmatched ground truth annotations
                annotations[filename].append(
                    [filename, img_width, img_height, gt_obj_id] + gt_box + 
                    [None, None, None, None, None, None, None] +  # Blank columns for score_all
                    [None] * len(pred_data[0]['score_all'])  # Ensure correct number of blank entries
                )

        # Process unmatched predictions
        for pred_obj in pred_annotations:
            # Check if prediction is not already matched using a similar approach
            pred_tuple = (
                pred_obj['category_id'], 
                tuple(pred_obj['bbox']),
                pred_obj.get('score', None),
                tuple(pred_obj['score_all'])
            )
            if pred_tuple not in matched_predictions:
                annotations[filename].append(
                    [filename, img_width, img_height, None, None, None, None, None] + 
                    [pred_obj['category_id']] + pred_obj['bbox'] + 
                    [pred_obj.get('score', None)] + list(pred_obj['score_all'])  # Add score_all values
                )

    return annotations

# Function to write annotations to CSV
def write_to_csv(annotations, output_csv):
    with open(output_csv, "w", newline="") as csvfile:
        csv_writer = csv.writer(csvfile)
        headers = [
            "filename", "image_width", "image_height", "gt_obj_id", "gt_x", "gt_y", "gt_width", "gt_height", 
            "pred_obj_id", "pred_x", "pred_y", "pred_width", "pred_height", "conf_score"
        ]
        
        # Add headers for score_all columns dynamically
        num_classes = len(next(iter(annotations.values()))[0]) - len(headers)
        for i in range(num_classes):
            headers.append(f"class_{i}_conf_score")
        
        csv_writer.writerow(headers)
        
        for filename, ann_list in annotations.items():
            for ann in ann_list:
                csv_writer.writerow(ann)

# Define paths and files based on provided information
gt_file = "valid.json"
prediction_file = "faster_rcnn.json"
output_csv = "faster_rcnn.csv"

# Process files and write annotations to CSV
annotations = process_files(gt_file, prediction_file)
write_to_csv(annotations, output_csv)